# Aggregating MFT to the county level

In order to merge the MFT data (recorded at the zipcode level) with the IRS/ACS data (recorded at the county level), it is necessary to get convert from zip code to county as the summary level. To do this, we will use the Census' [zipcode to county relationship file](https://www.census.gov/geographies/reference-files/time-series/geo/relationship-files.2010.html).

In [1]:
import pandas as pd

### Merging relationship file and MFT data

In [2]:
rel_df = pd.read_csv("./data/zcta_county_rel_10.csv", dtype={'ZCTA5': object})
mft_df = pd.read_csv("./data/mft_county_rel.csv", dtype={'ZCTA5': object})

In [3]:
mft_county_rel = pd.merge(rel_df, mft_df, how='left')

Checking that we didn't leave anything behind, by looking for any rows where the zip code was in the MFT dataset but the United Way field is empty. There shouldn't be any rows where this is true.

In [4]:
mft_county_rel[mft_county_rel.ZCTA5.isin(mft_df.ZCTA5) & pd.isnull(mft_county_rel['United Way'])]

,ZCTA5,STATE,COUNTY,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,...,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT,City,State,County,Org ID,United Way,#e-filed returns


Everything looks good!

In [5]:
mft_county_rel.columns

Index(['ZCTA5', 'STATE', 'COUNTY', 'GEOID', 'POPPT', 'HUPT', 'AREAPT',
       'AREALANDPT', 'ZPOP', 'ZHU', 'ZAREA', 'ZAREALAND', 'COPOP', 'COHU',
       'COAREA', 'COAREALAND', 'ZPOPPCT', 'ZHUPCT', 'ZAREAPCT', 'ZAREALANDPCT',
       'COPOPPCT', 'COHUPCT', 'COAREAPCT', 'COAREALANDPCT', 'City', 'State',
       'County', 'Org ID', 'United Way', '#e-filed returns'],
      dtype='object')

### Aggregating to the county

Because zipcodes and counties are not co-terminous, some zipcodes are in multiple counties. The column `ZPOPPCT` in the zipcode to county relationship file denotes how much of the zipcode's population is in each county. For example, zip code `39573` stretches across 5 counties:

In [6]:
mft_county_rel[mft_county_rel.ZCTA5 == '39573'][['ZCTA5', 'City', 'State', 'County', 'STATE', 'COUNTY', 'ZPOPPCT']]

,ZCTA5,City,State,County,STATE,COUNTY,ZPOPPCT
16570,39573,Perkinston,MS,Stone,28,39,3.56
16571,39573,Perkinston,MS,Stone,28,45,21.91
16572,39573,Perkinston,MS,Stone,28,47,1.11
16573,39573,Perkinston,MS,Stone,28,59,1.52
16574,39573,Perkinston,MS,Stone,28,109,1.31
16575,39573,Perkinston,MS,Stone,28,131,70.59


We want one county per zipcode, so we'll take only the row with the highest percentage of people. In the case of zipcode `38573`, that would be the last row (the one with a county FIPS code of `131`).

In [7]:
mft_county_rel = mft_county_rel.sort_values(['ZCTA5', 'COUNTY', 'ZPOPPCT']).drop_duplicates(['ZCTA5', 'COUNTY'], keep='last')

In [8]:
mft_county_rel.describe()

,STATE,COUNTY,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,ZAREA,...,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT,#e-filed returns
count,44409.000000,44409.000000,44409.000000,44409.000000,44409.000000,4.440900e+04,4.440900e+04,44409.000000,44409.000000,4.440900e+04,...,4.440900e+04,44409.000000,44409.000000,44409.000000,44409.000000,44409.000000,44409.000000,44409.000000,44409.000000,21906.000000
mean,30.124142,87.833637,30211.975140,7036.027607,3002.365939,1.711418e+08,1.673306e+08,9111.244207,3901.043572,2.626092e+08,...,3.466145e+09,74.255175,73.894835,74.578953,74.578992,7.252251,7.248556,6.705254,6.923092,12.697982
std,15.323056,85.813008,15333.111889,12341.371141,5040.099625,5.626424e+08,5.504662e+08,13235.141549,5399.305229,6.597467e+08,...,1.220071e+10,38.601151,38.688594,35.947392,35.956118,14.184957,13.916541,10.568559,10.867822,22.525799
min,1.000000,1.000000,1001.000000,0.000000,0.000000,2.270000e+03,2.270000e+03,0.000000,0.000000,5.094000e+03,...,5.176813e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,18.000000,31.000000,18127.000000,277.000000,139.000000,1.692736e+07,1.606413e+07,867.000000,432.000000,3.982343e+07,...,1.214167e+09,42.560000,40.490000,45.980000,46.040000,0.610000,0.650000,0.830000,0.890000,2.000000
50%,29.000000,71.000000,29221.000000,1324.000000,635.000000,6.267203e+07,6.080415e+07,2968.000000,1390.000000,1.249640e+08,...,1.727139e+09,100.000000,100.000000,100.000000,100.000000,2.230000,2.330000,3.120000,3.300000,6.000000
75%,42.000000,119.000000,42063.000000,7345.000000,3222.000000,1.641167e+08,1.596084e+08,11927.000000,5188.000000,2.794483e+08,...,2.562955e+09,100.000000,100.000000,100.000000,100.000000,6.550000,6.720000,8.120000,8.390000,13.000000
max,72.000000,840.000000,72153.000000,113916.000000,47617.000000,3.510842e+10,3.478610e+10,113916.000000,47617.000000,3.510842e+10,...,3.768557e+11,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,456.000000


And, since some counties have lots of zipcodes, we need to get a total per county.

In [9]:
mft_per_county = mft_county_rel.groupby(['STATE', 'COUNTY']).agg({'#e-filed returns': 'sum'}).reset_index()

Making sure all of the counties made it to `mft_county_rel`.

In [10]:
len(mft_county_rel.groupby(['STATE', 'COUNTY']).groups) == len(rel_df.groupby(['STATE', 'COUNTY']).groups)

True

In [11]:
mft_per_county.to_csv("./data/mft_at_county_level.csv", index=False)